In [11]:
import sys
import pandas as pd

## Read data from fix width formatted file

In [12]:
sys.path.insert(0, '..')
from Modules.read_fwf import ReadStataDct

In [13]:
def ReadFemPreg(dct_file, dat_file):
    """
    reads the NSFG pregnancy data.

    :param dct_file: string file name
    :param dat_file: string file name
    :return: DataFrame
    """
    dct = ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file)

    return df

In [14]:
pregnancy_data = ReadFemPreg(dct_file='../Data/2017_2019_FemPregSetup.dct',
                             dat_file='../Data/2017_2019_FemPregData.dat')

In [15]:
pregnancy_data

,caseid,pregordr,moscurrp,pregend1,pregend2,hpageend,kidage,hpagelb,paybirth1,paybirth2,...,secu,sest,cmintvw,cmlstyr,cmjan3yr,cmjan4yr,cmjan5yr,quarter,phase,intvwyear
0,88819,1,NaN,6.0,NaN,NaN,7.0,4.0,NaN,NaN,...,2,354,1427,1415,1381,1369,1357,29,1,2018
1,88819,2,NaN,5.0,NaN,NaN,6.0,4.0,NaN,NaN,...,2,354,1427,1415,1381,1369,1357,29,1,2018
2,83055,1,NaN,5.0,NaN,NaN,6.0,4.0,NaN,NaN,...,2,354,1425,1413,1381,1369,1357,29,1,2018
3,83055,2,NaN,5.0,NaN,NaN,6.0,5.0,NaN,NaN,...,2,354,1425,1413,1381,1369,1357,29,1,2018
4,92062,1,NaN,6.0,NaN,NaN,7.0,4.0,NaN,NaN,...,2,354,1426,1414,1381,1369,1357,29,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10210,91420,4,NaN,6.0,NaN,NaN,6.0,6.0,NaN,NaN,...,1,370,1432,1420,1393,1381,1369,31,1,2019
10211,89157,1,NaN,6.0,NaN,NaN,6.0,3.0,NaN,NaN,...,1,370,1433,1421,1393,1381,1369,31,1,2019
10212,89157,2,NaN,6.0,NaN,NaN,6.0,4.0,NaN,NaN,...,1,370,1433,1421,1393,1381,1369,31,1,2019
10213,89157,3,NaN,5.0,NaN,NaN,5.0,4.0,1.0,2.0,...,1,370,1433,1421,1393,1381,1369,31,1,2019
